# **MIP3_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand."""


In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

1. Scheduling variables:
   - P_ij: Power output of the i-th thermal unit in the j-th time period 
   
2. Constant startup and shutdown costs:
   - SC_i: Startup cost of the i-th power unit
   - SD_i: Shutdown cost of the i-th power unit
   
3. Power status indicators:
   - ON_i: Indicates if the i-th power unit is turned on. If ON_i = 1, it means the power unit is turned on. If ON_i = 0, it means the power unit is turned off.
   - OFF_i: Indicates if the i-th power unit is turned off. If OFF_i = 1, it means the power unit is turned off. If OFF_i = 0, it means the power unit is turned on.

4. Variable costs:
   - VC_ij: Variable cost of the i-th power unit in the j-th time period

5. Output power bounds:
   - LB_i: Lower bound of the power output of the i-th power unit
   - UB_i: Upper bound of the power output of the i-th power unit 

6. Power increments and decrements:
   - INC_i: Maximum power increment of the i-th power unit
   - DEC_i: Maximum power decrement of the i-th power uni

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
print(response2.choices[0].message.content)

Minimize 

Objective Function: 

Sum[j=1 to 15] (Sum[i=1 to 6] ((SC_i * ON_ij + SD_i * OFF_ij + VC_ij * P_ij) * t_j )) 

where t_j represents the duration of each time period.


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [10]:
print(response3.choices[0].message.content)

1. Power output constraint: The power output of a power plant can not exceed its maximum and can not fall below its minimum:
   - LB_i ≤ P_ij ≤ UB_i, for all i and j 

2. Power increment constraint: The increment in power output can not exceed the maximum increment capacity of the power plant: 
   - P_ij - P_i(j-1) ≤ INC_i, for all i and j > 1 
  
3. Power decrement constraint: The decrement in power output can not exceed the maximum decrement capacity of the power plant:
   - P_i(j-1) - P_ij ≤ DEC_i, for all i and j > 1 

4. Power plant status constraint: 
   - ON_i + OFF_i = 1, for all i 
   
5. Power demand constraint: The sum of power outputs of all power plants should be equal to the power demand in that time period:
   - Sum[i=1 to 6] P_ij = D_j, for all j
  
6. Security margin constraint: Total available power should always be 10% higher than the demand:
   - D_j ≤ 0.9*Sum[i=1 to 6] P_ij, for all j



### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [12]:
Markdown(response4.choices[0].message.content)

Ok, let's create a Python Pyomo code for this optimization problem.

Here is a template for how this could be done using Python's Pyomo library.

```python
from pyomo.environ import *

# Initialize model
model = ConcreteModel()

# 1 is used where we need to represent sample data.

# Initialize indexes
model.I = range(1, 7) # index for power units
model.J = range(1, 16) # index for time periods

# Initialize parameters
model.SC = Param(model.I, initialize=1) # startup cost
model.SD = Param(model.I, initialize=1) # shutdown cost
model.VC = Param(model.I, model.J, initialize=1) # variable cost
model.LB = Param(model.I, initialize=1) # lower bound of power output
model.UB = Param(model.I, initialize=1) # upper bound of power output
model.INC = Param(model.I, initialize=1) # power increment capacity
model.DEC = Param(model.I, initialize=1) # power decrement capacity
model.D = Param(model.J, initialize=1) # power demand
model.S = Param(model.J, initialize=1) # security margin

# Initialize variables
model.P = Var(model.I, model.J, within=NonNegativeReals) # power output
model.ON = Var(model.I, within=Binary) # power unit status (on)
model.OFF = Var(model.I, within=Binary) # power unit status (off)

# Objective function
model.OBJ = Objective(expr = sum(sum((model.SC[i]*model.ON[i]+model.SD[i]*model.OFF[i]+model.VC[i,j]*model.P[i,j])*model.S[j] for i in model.I) for j in model.J), sense=minimize)

# Constraints
model.power_output_con = ConstraintList()
model.power_increment_con = ConstraintList()
model.power_decrement_con = ConstraintList()
model.power_status_con = ConstraintList()
model.power_demand_con = ConstraintList()
model.security_margin_con = ConstraintList()

for i in model.I:
	model.power_output_con.add(inequality(model.LB[i], model.P[i,j], model.UB[i]))
	model.power_status_con.add(model.ON[i] + model.OFF[i] == 1)
	for j in model.J:
		if j > 1:
			model.power_increment_con.add(model.P[i,j] - model.P[i,j-1] <= model.INC[i])
          		model.power_decrement_con.add(model.P[i,j-1] - model.P[i,j] <= model.DEC[i])
 		model.power_demand_con.add(sum(model.P[i,j] for i in model.I) == model.D[j])
 		model.security_margin_con.add(model.D[j] >= 0.9*sum(model.P[i,j] for i in model.I))

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)
```
Please note that this script is just a rough approximation and may not be precise due to lack of specific details. Feel free to modify the code according to your requirements. Also, replace the "initialize=1" with your actual data.

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# Initialize model
model = ConcreteModel()

# 1 is used where we need to represent sample data.

# Initialize indexes
model.I = range(1, 7) # index for power units
model.J = range(1, 16) # index for time periods

# Initialize parameters
model.SC = Param(model.I, initialize=1) # startup cost
model.SD = Param(model.I, initialize=1) # shutdown cost
model.VC = Param(model.I, model.J, initialize=1) # variable cost
model.LB = Param(model.I, initialize=1) # lower bound of power output
model.UB = Param(model.I, initialize=1) # upper bound of power output
model.INC = Param(model.I, initialize=1) # power increment capacity
model.DEC = Param(model.I, initialize=1) # power decrement capacity
model.D = Param(model.J, initialize=1) # power demand
model.S = Param(model.J, initialize=1) # security margin

# Initialize variables
model.P = Var(model.I, model.J, within=NonNegativeReals) # power output
model.ON = Var(model.I, within=Binary) # power unit status (on)
model.OFF = Var(model.I, within=Binary) # power unit status (off)

# Objective function
model.OBJ = Objective(expr = sum(sum((model.SC[i]*model.ON[i]+model.SD[i]*model.OFF[i]+model.VC[i,j]*model.P[i,j])*model.S[j] for i in model.I) for j in model.J), sense=minimize)

# Constraints
model.power_output_con = ConstraintList()
model.power_increment_con = ConstraintList()
model.power_decrement_con = ConstraintList()
model.power_status_con = ConstraintList()
model.power_demand_con = ConstraintList()
model.security_margin_con = ConstraintList()

for i in model.I:
    model.power_output_con.add(inequality(model.LB[i], model.P[i,j], model.UB[i]))
    model.power_status_con.add(model.ON[i] + model.OFF[i] == 1)
    for j in model.J:
        if j > 1:
            model.power_increment_con.add(model.P[i,j] - model.P[i,j-1] <= model.INC[i])
                  model.power_decrement_con.add(model.P[i,j-1] - model.P[i,j] <= model.DEC[i])
         model.power_demand_con.add(sum(model.P[i,j] for i in model.I) == model.D[j])
         model.security_margin_con.add(model.D[j] >= 0.9*sum(model.P[i,j] for i in model.I))

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 46)

In [21]:
from pyomo.environ import *

# Initialize model
model = ConcreteModel()

# 1 is used where we need to represent sample data.

# Initialize indexes
model.I = range(1, 7) # index for power units
model.J = range(1, 16) # index for time periods

# Initialize parameters
model.SC = Param(model.I, initialize=1) # startup cost
model.SD = Param(model.I, initialize=1) # shutdown cost
model.VC = Param(model.I, model.J, initialize=1) # variable cost
model.LB = Param(model.I, initialize=1) # lower bound of power output
model.UB = Param(model.I, initialize=1) # upper bound of power output
model.INC = Param(model.I, initialize=1) # power increment capacity
model.DEC = Param(model.I, initialize=1) # power decrement capacity
model.D = Param(model.J, initialize=1) # power demand
model.S = Param(model.J, initialize=1) # security margin

# Initialize variables
model.P = Var(model.I, model.J, within=NonNegativeReals) # power output
model.ON = Var(model.I, within=Binary) # power unit status (on)
model.OFF = Var(model.I, within=Binary) # power unit status (off)

# Objective function
model.OBJ = Objective(expr = sum(sum((model.SC[i]*model.ON[i]+model.SD[i]*model.OFF[i]+model.VC[i,j]*model.P[i,j])*model.S[j] for i in model.I) for j in model.J), sense=minimize)

# Constraints
model.power_output_con = ConstraintList()
model.power_increment_con = ConstraintList()
model.power_decrement_con = ConstraintList()
model.power_status_con = ConstraintList()
model.power_demand_con = ConstraintList()
model.security_margin_con = ConstraintList()

for i in model.I:
  for j in model.J:
        if j > 1:
            model.power_increment_con.add(model.P[i,j] - model.P[i,j-1] <= model.INC[i])
            model.power_decrement_con.add(model.P[i,j-1] - model.P[i,j] <= model.DEC[i])
            model.power_demand_con.add(sum(model.P[i,j] for i in model.I) == model.D[j])
            model.security_margin_con.add(model.D[j] >= 0.9*sum(model.P[i,j] for i in model.I))
  model.power_output_con.add(inequality(model.LB[i], model.P[i,j], model.UB[i]))
  model.power_status_con.add(model.ON[i] + model.OFF[i] == 1)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 348, 'Number of variables': 102, 'Number of nonzeros': 1362, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'infeasible', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.007287502288818359}]}

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**